## FireFly For Tuning HyperParameters

In [218]:
import torch, torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

import numpy as np
from dataclasses import dataclass
import matplotlib.pyplot as plt
import random
import os

In [225]:
torch.use_deterministic_algorithms(True)

In [350]:
class FireflyAlgorithm:
    def __init__(self, pop_size=20, alpha=1.0, betamin=1.0, gamma=0.01, seed=None):
        self.pop_size = pop_size
        self.alpha = alpha
        self.betamin = betamin
        self.gamma = gamma
        self.rng = np.random.default_rng(seed)

    def run(self, function, dim, lb=-5, ub=5, max_evals=100):
        fireflies = self.rng.uniform(lb, ub, (self.pop_size, dim))
        intensity = np.apply_along_axis(function, 1, fireflies)
        self.best_intensity = np.min(intensity)
        self.best_pos = fireflies[np.argmin(intensity)]

        evaluations = self.pop_size
        new_alpha = self.alpha
        search_range = ub - lb

        while evaluations <= max_evals:
            new_alpha *= 0.97
            for i in range(self.pop_size):
                for j in range(self.pop_size):
                    if intensity[i] >= intensity[j] and not np.isnan(intensity[j]):  # Check for np.nan
                        r = np.sum(np.square(fireflies[i] - fireflies[j]), axis=-1)
                        beta = self.betamin * np.exp(-self.gamma * r)
                        steps = new_alpha * (self.rng.random(dim) - 0.5) * search_range
                        fireflies[i] += beta * (fireflies[j] - fireflies[i]) + steps
                        fireflies[i] = np.clip(fireflies[i], lb, ub)
                        intensity[i] = function(fireflies[i])
                        evaluations += 1
                        if not np.isnan(intensity[i]) and intensity[i] < self.best_intensity:  # Check for np.nan
                            self.best_intensity = intensity[i]
                            self.best_pos = fireflies[i].copy()


In [351]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

housing = fetch_california_housing()

In [352]:
data = housing
X = torch.Tensor(data.data)
y = torch.Tensor(data.target)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Build a simple Linear model:

In [353]:
train_set = TensorDataset(X_train, y_train.unsqueeze(0).t())
test_set = TensorDataset(X_test, y_test.unsqueeze(0).t())

In [354]:
train_loader = DataLoader(train_set, shuffle=True, batch_size=64)
test_loader = DataLoader(test_set, shuffle=True, batch_size=64)

In [355]:
def train(params):

    lr = float(params[0]/20)
    hidden = int(params[1])
    
    model = nn.Sequential(
                nn.Linear(8, hidden),
                nn.ReLU(True),
                nn.Linear(hidden, 1)
            )
    
    criterion = nn.MSELoss()
    opt = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(1):

        for x, y in train_loader:

            opt.zero_grad()

            y_pred = model(x)

            loss = criterion(y_pred, y)

            loss.backward()

            opt.step()

    mse = 0
    
    with torch.no_grad():    
        for x, y in test_loader:
            outputs = model(x)
            mse += criterion(y, outputs)
    
    return (mse / len(test_loader)).item()

In [356]:
FA = FireflyAlgorithm()

In [357]:
FA.run(function= train, dim= 2, lb= 0, ub= 10, max_evals= 10)

/home/moussa/.local/lib/python3.11/site-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [358]:
2.66252317/20

0.1331261585

In [359]:
FA.best_pos

array([9.01801454, 7.53344272])

In [360]:
FA.best_intensity

nan

In [361]:
FA.best_pos

array([9.01801454, 7.53344272])

In [362]:
9.56104393/20

0.4780521965

In [363]:
train([6.50232899, 9.04405793])

1.3143388032913208